# fact_messaging_time

### Importación de librerías

In [1]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

### Conexión a base y bodega de datos

In [2]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlFuente = f"{configFuente['driver']}://{configFuente['user']}:{configFuente['password']}@{configFuente['host']}:{configFuente['port']}/{configFuente['db']}"
urlBodega = f"{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}"

src = create_engine(urlFuente)
etl = create_engine(urlBodega)

### Extracción y transformación de datos

In [3]:
mensajeria_servicio = pd.read_sql_table('mensajeria_servicio', src)
mensajeria_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', src)
clientes_mensajeroaquitoy = pd.read_sql_table('clientes_mensajeroaquitoy', src)

mensajeria_servicio.drop(columns=["descripcion", "nombre_solicitante", "fecha_solicitud", "hora_solicitud", "fecha_deseada", "hora_deseada", "nombre_recibe", "telefono_recibe", "descripcion_pago", "ida_y_regreso", "activo", "novedades", "cliente_id", "destino_id", "origen_id", "tipo_pago_id", "tipo_servicio_id", "tipo_vehiculo_id", "usuario_id", "prioridad", "ciudad_destino_id", "ciudad_origen_id", "hora_visto_por_mensajero", "visto_por_mensajero", "descripcion_multiples_origenes", "multiples_origenes", "asignar_mensajero", "es_prueba", "descripcion_cancelado"], inplace=True)
mensajeria_estadosservicio.drop(columns=["fecha", "foto", "observaciones", "es_prueba", "foto_binary"], inplace=True)
clientes_mensajeroaquitoy.drop(columns=["activo", "fecha_entrada", "fecha_salida", "salario", "telefono", "ciudad_operacion_id", "token_Firebase", "url_foto"], inplace=True)

mensajeria_servicio['mensajero2_id'].fillna(mensajeria_servicio['mensajero_id'], inplace=True)
mensajeria_servicio['mensajero3_id'].fillna(mensajeria_servicio['mensajero2_id'], inplace=True)

mensajeria_estadosservicio = mensajeria_estadosservicio[mensajeria_estadosservicio['estado_id'] == 2]

hecho_entrega_mensajeria_hora_nm1 = pd.merge(mensajeria_servicio, mensajeria_estadosservicio, left_on='id', right_on='servicio_id', how='inner')
hecho_entrega_mensajeria_hora = pd.merge(hecho_entrega_mensajeria_hora_nm1, clientes_mensajeroaquitoy, left_on='mensajero3_id', right_on='id', how='inner')

registro_de_hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros = []
conteo_de_mensajeros_por_servicio = {}
for index, row in hecho_entrega_mensajeria_hora.iterrows():
    servicio_id = hecho_entrega_mensajeria_hora.iloc[index, 7]
    if servicio_id not in conteo_de_mensajeros_por_servicio:
        conteo_de_mensajeros_por_servicio[servicio_id] = 1
    elif conteo_de_mensajeros_por_servicio[servicio_id] == 1:
        conteo_de_mensajeros_por_servicio[servicio_id] += 1
    elif conteo_de_mensajeros_por_servicio[servicio_id] == 2:
        nueva_fila = {'hora': hecho_entrega_mensajeria_hora.iloc[index, 5].replace(minute=0, second=0), 'key_messenger': int(hecho_entrega_mensajeria_hora.iloc[index, 9])}
        registro_de_hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros.append(nueva_fila)
        conteo_de_mensajeros_por_servicio[servicio_id] += 1

hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros = pd.DataFrame(registro_de_hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros)
hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupados = hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros.groupby(['key_messenger', 'hora']).size().reset_index(name='total_hora')
hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupados["key_hecho_entrega_mensajeria_hora"] = range(1, len(hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupados) + 1)

dim_hora = pd.read_sql_table('dim_hora', etl)

hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupadoss_con_hora_id = hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupados.merge(dim_hora[['hora', 'key_dim_hora']], on='hora', how='left')
hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupadoss_con_hora_id.drop(columns=['hora'], inplace=True)
hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupadoss_con_hora_id.rename(columns={'key_dim_hora': 'key_hora'}, inplace=True)

hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupadoss_con_hora_id

C:\Users\jpant\AppData\Local\Temp\ipykernel_20364\427345242.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mensajeria_servicio['mensajero2_id'].fillna(mensajeria_servicio['mensajero_id'], inplace=True)
C:\Users\jpant\AppData\Local\Temp\ipykernel_20364\427345242.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are sett

,key_messenger,total_hora,key_hecho_entrega_mensajeria_hora,key_hora
0,1,1,1,121
1,1,1,2,421
2,1,1,3,481
3,1,1,4,661
4,1,1,5,721
...,...,...,...,...
171,447,3,172,601
172,447,1,173,841
173,447,2,174,901
174,447,1,175,961


### Carga de datos

In [4]:
hecho_entrega_mensajeria_hora_con_tiempos_y_mensajeros_agrupadoss_con_hora_id.to_sql("hecho_entrega_mensajeria_hora", etl, index=False, if_exists="replace")

176